In [89]:
import os
import requests

In [90]:
r = requests.get('https://usegalaxy.org/api/tools')

In [91]:
usegalaxy_tools = r.json()

In [92]:
def get_chunks(l, size):
    for i in xrange(0, len(l), size):
        yield l[i: i + size]

In [99]:
dockerfile = """
# Galaxy - usegalaxy.org clone
#
# VERSION       0.1

FROM bgruening/galaxy-stable:dev

MAINTAINER Bjoern A. Gruening, bjoern.gruening@gmail.com

ENV GALAXY_CONFIG_BRAND usegalaxy.org Clone


"""               

repositories = list()
for section in usegalaxy_tools:
    section_name = section.get('name', '').replace(' ', '_')
    if 'elems' not in section:
        continue
    for tool in section['elems']:
        idd = tool['id']
        if 'toolshed' in idd:
            items = idd.split('/')
            ts = items[0]
            owner = items[2]
            name = items[3]
            repo = '''    "--url https://%s -o %s --name %s --panel-section-name '%s'"''' % (ts, owner, name, section_name)
            repositories.append(repo)

for repos in get_chunks(list(set(repositories)), 10):
    dockerfile += "RUN install-repository \\\n"
    dockerfile += " \\\n".join(repos)
    dockerfile += "\n\n"
with open(os.path.join(os.path.expanduser('~'), 'usegalaxy.Dockerfile'), 'w+') as handle:
    handle.write(dockerfile)
print(dockerfile)


# Galaxy - usegalaxy.org clone
#
# VERSION       0.1

FROM bgruening/galaxy-stable:dev

MAINTAINER Bjoern A. Gruening, bjoern.gruening@gmail.com

ENV GALAXY_CONFIG_BRAND usegalaxy.org Clone


RUN install-repository \
    "--url https://toolshed.g2.bx.psu.edu -o devteam --name subtract_query --panel-section-name 'Join,_Subtract_and_Group'" \
    "--url https://toolshed.g2.bx.psu.edu -o devteam --name indel_realigner --panel-section-name 'NGS:_GATK_Tools_(beta)'" \
    "--url https://toolshed.g2.bx.psu.edu -o devteam --name table_recalibration --panel-section-name 'NGS:_GATK_Tools_(beta)'" \
    "--url https://toolshed.g2.bx.psu.edu -o anton --name vcfallelicprimitives --panel-section-name 'NGS:_VCF_Manipulation'" \
    "--url https://toolshed.g2.bx.psu.edu -o devteam --name picard --panel-section-name 'NGS:_Picard_(beta)'" \
    "--url https://toolshed.g2.bx.psu.edu -o devteam --name filter_transcripts_via_tracking --panel-section-name 'NGS:_RNA-seq'" \
    "--url https://toolshed.g2.b